# 내부 단어(subword)의 학습

In [5]:
# n = 3인 경우
['<ap', 'app', 'ppl', 'ple', 'le>']

['<ap', 'app', 'ppl', 'ple', 'le>']

In [4]:
# 특별 토큰


'<apple>'

In [6]:
# n = 3인 경우
['<ap', 'app', 'ppl', 'ple', 'le>','<apple>']

['<ap', 'app', 'ppl', 'ple', 'le>', '<apple>']

n = 3 ~ 6인 경우

<ap, app, ppl, ppl, le>, <app, appl, pple, ple>, <appl, pple>, ..., \<apple>

단어 apple의 벡터값은 저 위 벡터값들의 총 합으로 구성합니다.

# 실습으로 비교하는 Word2Vec Vs. FastText

In [9]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Word2Vec

In [11]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format("/gdrive/My Drive/Colab Notebooks/study NLP/model/eng_w2v")

In [13]:
# electrofishing가 단어 집합에 없으므로 에러가 발생
word2vec.wv.most_similar("electrofishing")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


KeyError: ignored

## FastText

In [14]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

### 훈련 데이터 이해하기

In [15]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fb9d30eb1d0>)

### 훈련 데이터 전처리하기

In [16]:
import nltk
nltk.download('punkt')

targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [17]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [18]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


### FastText 훈련시키기

In [19]:
from gensim.models import FastText

model = FastText(sentences=result, size=100, window=5, min_count=5, workers=4, sg=1)

In [20]:
model_result = model.wv.most_similar("electrofishing")
print(model_result)

[('electrolux', 0.8043632507324219), ('electrolyte', 0.7979683876037598), ('electro', 0.794302225112915), ('electroencephalogram', 0.7788436412811279), ('electrogram', 0.7733550071716309), ('electric', 0.7723609209060669), ('airbus', 0.7670558094978333), ('electrochemical', 0.7643627524375916), ('electroshock', 0.7639851570129395), ('electron', 0.75522780418396)]
